Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [75]:
print(df.shape)
df.head()

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [0]:
#Okay subsetting data.
#Doing a copy, because I'm going to use inplace later
#so first splitting to single family dwellings
subset = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS'].copy()

#next price is 100,000 > 2,000,000
#lets try it with queries now
subset.query('SALE_PRICE > 100000', inplace = True)
subset.query('SALE_PRICE < 2000000', inplace = True)

In [77]:
#Just verifying I properly subset my data
print(subset.shape)
print(subset['SALE_PRICE'].min())
print(subset['SALE_PRICE'].max())
subset['BUILDING_CLASS_CATEGORY'].value_counts()

(3151, 21)
104000
1955000


01 ONE FAMILY DWELLINGS    3151
Name: BUILDING_CLASS_CATEGORY, dtype: int64

In [78]:
#I should check for nans
subset.isnull().sum()

BOROUGH                              0
NEIGHBORHOOD                         0
BUILDING_CLASS_CATEGORY              0
TAX_CLASS_AT_PRESENT                 0
BLOCK                                0
LOT                                  0
EASE-MENT                         3151
BUILDING_CLASS_AT_PRESENT            0
ADDRESS                              0
APARTMENT_NUMBER                  3150
ZIP_CODE                             0
RESIDENTIAL_UNITS                    0
COMMERCIAL_UNITS                     0
TOTAL_UNITS                          0
LAND_SQUARE_FEET                     0
GROSS_SQUARE_FEET                    0
YEAR_BUILT                           0
TAX_CLASS_AT_TIME_OF_SALE            0
BUILDING_CLASS_AT_TIME_OF_SALE       0
SALE_PRICE                           0
SALE_DATE                            0
dtype: int64

In [79]:
#Okay, easment is a garbage column so I can drop that, and apartment is more or
#less a garbage column with only one value.  So lets just kill those.
subset.drop(['EASE-MENT', 'APARTMENT_NUMBER'], axis = 1, inplace = True)
print(subset.shape)
subset.isnull().sum()

(3151, 19)


BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [80]:
#turning sale date into datetime object for better analysis
subset['DATE_TIME'] = pd.to_datetime(subset['SALE_DATE'])
print(subset['DATE_TIME'].min())
print(subset['DATE_TIME'].max())

2019-01-01 00:00:00
2019-04-30 00:00:00


In [0]:
#okay easy enough, just split on the first of april
train = subset[subset['DATE_TIME'] < pd.Timestamp('04/01/2019')]
test = subset[subset['DATE_TIME'] >= pd.Timestamp('04/01/2019')]

In [82]:
print(train.shape)
train['DATE_TIME'].describe()

(2507, 20)


count                    2507
unique                     68
top       2019-01-31 00:00:00
freq                       78
first     2019-01-01 00:00:00
last      2019-03-30 00:00:00
Name: DATE_TIME, dtype: object

In [83]:
print(test.shape)
test['DATE_TIME'].describe()

(644, 20)


count                     644
unique                     23
top       2019-04-15 00:00:00
freq                       43
first     2019-04-01 00:00:00
last      2019-04-30 00:00:00
Name: DATE_TIME, dtype: object

In [89]:
#okay looks like training and testing sets are split like I wanted them to
#next I need to drop a few non-categorical columns for the actual training sets
#otherwise I get some wacky OneHotEncoding (like a category for every date
#in the data set).  

#So first lets just eyeball it and drop anything that I know will be a problem
subset.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE,DATE_TIME
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,1260 RHINELANDER AVE,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,469 E 25TH ST,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,5521 WHITTY LANE,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019,2019-01-02


In [91]:
#trying to identify high cardinality features
for column in subset.columns:
  print(subset[column].value_counts())

4    1580
5     738
3     537
2     293
1       3
Name: BOROUGH, dtype: int64
OTHER                 2990
FLUSHING-NORTH          97
FOREST HILLS            22
BOROUGH PARK            19
ASTORIA                 14
BEDFORD STUYVESANT       9
Name: NEIGHBORHOOD, dtype: int64
01 ONE FAMILY DWELLINGS    3151
Name: BUILDING_CLASS_CATEGORY, dtype: int64
1     3111
1D      40
Name: TAX_CLASS_AT_PRESENT, dtype: int64
16350    21
1272      7
5506      6
6022      6
5735      6
         ..
569       1
4667      1
5730      1
6720      1
6145      1
Name: BLOCK, Length: 2496, dtype: int64
20      58
19      56
1       53
31      47
40      46
        ..
556      1
554      1
488      1
484      1
1200     1
Name: LOT, Length: 332, dtype: int64
A1    1185
A5     985
A2     491
A9     239
A0      85
S1      48
A3      43
A8      40
A4      16
A6      15
B2       2
B3       1
S0       1
Name: BUILDING_CLASS_AT_PRESENT, dtype: int64
33 BAILEY PLACE              2
216-29 114TH ROAD            2
118-20 

In [101]:
#removing all high cardinality columns, and YEAR_BUILT because it only has one
#value thus is meaningless. I left zipcode and year built in, because IDK they
#seem like they might actually be interesting categories, and they aren't HUGE
#but I might regret that.
to_drop = ['ADDRESS', 'SALE_DATE', 'DATE_TIME', 'SALE_PRICE', 'YEAR_BUILT', 
           'GROSS_SQUARE_FEET', 'LAND_SQUARE_FEET', 'LOT', 'BLOCK']

#lets do these before we drop sales prices
y_train = train['SALE_PRICE']
y_test = test['SALE_PRICE']


X_train = train.drop(to_drop, axis = 1)
X_test = test.drop(to_drop, axis = 1)

print(X_train.shape)
print(X_train.columns)
print(X_test.shape)
print(X_test.columns)

(2507, 11)
Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')
(644, 11)
Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')


In [102]:
#looking good, lets go ahead and encode
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names = True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

print(X_train.shape)
print(X_test.shape)

(2507, 43)
(644, 43)


In [110]:
#Now I'm checking for anything with a STD of 0.  Because it means that all rows
#in the dataset, have the exact same value (no deviation from the mean) and are
#useless for evaluation.  This step is particularly important for SelectKBest
#which will throw warnings for anything with a 0 std.
X_train.std()

BOROUGH_3                                            0.365521
BOROUGH_4                                            0.499710
BOROUGH_2                                            0.295375
BOROUGH_5                                            0.440920
BOROUGH_1                                            0.019972
NEIGHBORHOOD_OTHER                                   0.217700
NEIGHBORHOOD_FLUSHING-NORTH                          0.172576
NEIGHBORHOOD_BEDFORD STUYVESANT                      0.056411
NEIGHBORHOOD_FOREST HILLS                            0.082084
NEIGHBORHOOD_BOROUGH PARK                            0.069033
NEIGHBORHOOD_ASTORIA                                 0.066108
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS      0.000000
TAX_CLASS_AT_PRESENT_1                               0.110532
TAX_CLASS_AT_PRESENT_1D                              0.110532
BUILDING_CLASS_AT_PRESENT_A9                         0.266620
BUILDING_CLASS_AT_PRESENT_A1                         0.481965
BUILDING

In [111]:
#Dropping the following for having 0 std, which means every value in the column
#is the same, and not useful for analysis.

std_0 = ['BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS', 
         'TAX_CLASS_AT_TIME_OF_SALE']

X_train.drop(std_0, axis = 1, inplace = True)
X_test.drop(std_0, axis = 1, inplace = True)
X_train.shape, X_test.shape

((2507, 41), (644, 41))

In [112]:
#okay lets us SelectKBest to pick 15 features that best correlate with the
#target. I selected 15 features as just an arbitrary number.  As a stretch I 
#will try to select the best set of features with the lowest mae.

from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func = f_regression, k = 15)

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

print(X_train_selected.shape)
print(X_test_selected.shape)

(2507, 15)
(644, 15)


In [113]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

model = Ridge(alpha = 10, normalize = True)

model.fit(X_train_selected, y_train)

y_pred = model.predict(X_test_selected)

mae = mean_absolute_error(y_test, y_pred)

print('Mean absolute error for my MVP model is: $', mae)

Mean absolute error for my MVP model is: $ 207069.39660609598


#Stretch goal
Find the best combination of features that generates the lowest mae.